# Market Analysis of C-UAS products

A comprehensive database [^Michel, A. H. (2018). Counter-drone systems. Center for the Study of the Drone at Bard College.] of publicly known counter-drone systems has been analyzed. The database consists of 235 products sold by 155 firms and partnerships. 

In [10]:
import pandas as pd
import numpy as np
import scipy as sp
import chart_studio.plotly as py
import plotly.figure_factory as ff
import plotly.graph_objects as go
import plotly.express as px

## Database of C-UAS products

As can be seen from the table on the Appendix, each solution reports the number of detection and interdiction options they use as well as the platforms on which they are installed. 

Many systems integrate a variety of different sensor types for more robust detection capability and also employ a combination of interdiction elements. 

All these combinations is studied in depth in the next sections.

In [11]:
df = pd.read_csv("CSD_Counter_Drone_Systems.csv")

In [12]:
detection = df[['FR','RADAR','EO','IR','ACUST','ALTRI']]
interdiction = df[['SPOOFING','RF_Jamming','GNSS_Jamming','NET','LASER','ALTRO']]
platform = df[['GROUND_BASED','UAV','ALTRO']]
manuf = df['MANUFACTURER']

df_detection = pd.DataFrame(detection.count(axis = 1))
df_interdiction = pd.DataFrame(interdiction.count(axis = 1))
df_platform = pd.DataFrame(platform.count(axis = 1))

table = df[['MANUFACTURER','PRODUCT','COUNTRY']]
table['DETECTION OPT'] = df_detection 
table['INTERDICTION OPT'] = df_interdiction 
table['PLATFORM OPT'] = df_platform 

As can be seen from the figure below, the counter-drone systems come from 33 different countries. The three most important are the USA, UK and Israel. 

From a European perspective, France, Germany and Switzerland are the countries where more solutions are manufactured.

In [13]:
Country = df.groupby('COUNTRY').size().reset_index(name='No: of SYSTEMS')
Country = Country.assign(iso_alpha = ['AUS','BLR','BEL','BRA','BGR','CDN','CHN','DNK','EST','FIN','FRA','DEU','HUN','IND','IRL','ISR','ITA','JPN','MCO','NLD','NOR','POL','RUS','SGP','ZAF','ESP','SWE','CHE','TWN','TUR','USA','GBR'])

sz = Country['No: of SYSTEMS'] * 10
fig = px.scatter_geo(Country, locations="iso_alpha", hover_name="COUNTRY",
                     color="No: of SYSTEMS", size = sz, projection="natural earth")
fig.show()

### Detection, Interdiction options and Platform types

From an overall analysis, it comes that eighty-eight (88) of the products in the database are designed only for detection, while 80 systems are intended solely for interdiction. In particular, hereunder, we are going to analyze each option: detection&tracking and interdiction, respectively.


#### Detection & Tracking options

As written above different C-UAS systems rely on a variety of techniques for detecting and/or intercepting drones, which can be summarised as following:

 - **Radar**: Detects the presence of small unmanned aircraft by their radar signature, which is generated when the aircraft encounters RF pulses emitted by the detection element. These systems often employ algorithms to distinguish between drones and other small, low-flying objects, such as birds.


 - **Radio-frequency (RF)**: Identifies the presence of drones by scanning for the frequencies on which most drones are known to operate. Algorithms pick out and geo-locate RF-emitting devices in the area that are likely to be drones.


 - **Electro-Optical (EO)**: Detects drones based on their visual signature.


 - **Infrared (IR)**: Detects drones based on their heat signature.


 - **Acoustic**: Detects drones by recognizing the unique sounds produced by their motors. Acoustic systems rely on a library of sounds produced by known drones, which are then matched to sounds detected in the operating environment.


 - **Combined Sensors**: Many systems integrate a variety of different sensor types in order to provide a more robust detection capability. For example, a system might include an acoustic sensor that cues an optical camera when it detects a potential drone in the vicinity. The use of multiple detection elements may also be intended to increase the probability of a successful detection, given that no individual detection method is entirely failproof.

Once the database is analyzed some key data can be extracted:

 - *Of the 155 systems that are capable of detection, 95 appear to employ a single sensor type, while at least 60 employ a combination of one or more sensor types (we count EO and IR sensors as distinct detection elements, though the two are more often than not used in conjunction). Sixteen systems employ a combination of four or five different sensor types.*
 

 - As can be see from the figure below, Roughly an equal number of systems (approximately 60)  employ radar, RF detection, and EO sensors. Fifty six employ IR, while 18 systems employ an acoustic sensor. The remaining are other option

In [14]:
detection_FR = detection.groupby('FR').size().reset_index(name='No. of FR')
detection_FR.columns = ['D.Opt', 'Num']
detection_RADAR = detection.groupby('RADAR').size().reset_index(name='No. of RADAR')
detection_RADAR.columns = ['D.Opt', 'Num']
detection_EO = detection.groupby('EO').size().reset_index(name='No. of EO')
detection_EO.columns = ['D.Opt', 'Num']
detection_IR = detection.groupby('IR').size().reset_index(name='No. of IR')
detection_IR.columns = ['D.Opt', 'Num']
detection_ACUST = detection.groupby('ACUST').size().reset_index(name='No. of ACUST')
detection_ACUST.columns = ['D.Opt', 'Num']
detection_ALTRI = detection['ALTRI'].count()
df_app_ALTRI = pd.DataFrame({'D.Opt': ['Altro']})
df_app_ALTRI['Num'] = {detection_ALTRI}

df_app = detection_FR
df_app2 = df_app.append(detection_RADAR)
df_app3 = df_app2.append(detection_EO)
df_app4 = df_app3.append(detection_IR)
df_app5 = df_app4.append(detection_ACUST)
df_app6 = df_app5.append(df_app_ALTRI)

fig = px.bar(df_app6, x='D.Opt', y='Num')

fig.update_layout(
    title="CSD - Counter Drone Systems // **Detection & Tracking Systems**",
    xaxis_title="Detection Solution",
    yaxis_title="num. of Systems",
    font=dict(
        family="Courier New, monospace",
        size=14,
        color="#7f7f7f"
    )
)

fig.show()

#### Interdiction options

As for detection, also in the case of interdiction the C-UAS systems rely on a variety of techniques, which can be summarised as following:

- **RF Jamming**: Disrupts the radio frequency link between the drone and its operator by generating large volumes of RF output. Once the RF link, which can include WiFi links, is severed, a drone will either descend to the ground or initiate a “return to home” maneuver.


- **GNSS Jamming**: Disrupts the drone’s satellite link, such as GPS or GLONASS, which is used for navigation. Drones that lose their satellite link will hover in place, land, or return to home.


- **Spoofing**: Allows one to take control of the targeted drone by hijacking the drone’s communications link. (Also known as protocol manipulation.)


- **Laser**: Destroys vital segments of the drone’s airframe using directed energy, causing it to crash to the ground.


- **Nets**: Designed to entangle the targeted drone and/or its rotors.


- **Projectile**: Employs regular or custom-designed ammunition to destroy incoming unmanned aircraft.


- **Combined Interdiction**: A number of C-UAS systems also employ a combination of interdiction elements

Once the database is analyzed some key data can be extracted:

 - About half of the 147 systems capable of interdiction rely on a single technique and half rely on two or more techniques (we count RF and GNSS jamming systems as distinct, even though the two are more often than not used in conjunction).
 
 - Jamming (Both RF and GNSS) is the most common interdiction method among systems: 
     - Jamming (RF, GNSS, or Both) 96
     - Net 17
     - Laser 10
     - Other 10
     
  - 88 systems depend on some form of signal jamming alone, while an additional eight systems employ jamming along with another technique. 
  
  
 - Approximately 30 systems rely solely on kinetic means to intercept and destroy targeted unmanned aircraft, while five systems feature both jamming and kinetic elements. Twelve systems have a spoofing capability.

In [15]:
interdiction_SPOOFING = interdiction.groupby('SPOOFING').size().reset_index(name='No. of SPOOFING')
interdiction_SPOOFING.columns = ['D.Opt', 'Num']
interdiction_RF_Jamming = interdiction.groupby('RF_Jamming').size().reset_index(name='No. of RF_Jamming')
interdiction_RF_Jamming.columns = ['D.Opt', 'Num']
interdiction_GNSS_Jamming = interdiction.groupby('GNSS_Jamming').size().reset_index(name='No. of GNSS_Jamming')
interdiction_GNSS_Jamming.columns = ['D.Opt', 'Num']
interdiction_NET = interdiction.groupby('NET').size().reset_index(name='No. of NET')
interdiction_NET.columns = ['D.Opt', 'Num']
interdiction_LASER = interdiction.groupby('LASER').size().reset_index(name='No. of LASER')
interdiction_LASER.columns = ['D.Opt', 'Num']
interdiction_ALTRO = interdiction['ALTRO'].count()
df_app_ALTRO = pd.DataFrame({'D.Opt': ['Altro']})
df_app_ALTRO['Num'] = {interdiction_ALTRO}

df_app = interdiction_SPOOFING
df_app2 = df_app.append(interdiction_RF_Jamming)
df_app3 = df_app2.append(interdiction_GNSS_Jamming)
df_app4 = df_app3.append(interdiction_NET)
df_app5 = df_app4.append(interdiction_LASER)
df_app6 = df_app5.append(df_app_ALTRO)
#df_app6

fig = px.bar(df_app6, x='D.Opt', y='Num')

fig.update_layout(
    title="CSD - Counter Drone Systems // **Interdiction Systems**",
    xaxis_title="Interdiction Solution",
    yaxis_title="num. of Systems",
    font=dict(
        family="Courier New, monospace",
        size=14,
        color="#7f7f7f"
    )
)

fig.show()

#### Platform Types

From the platfrom point of view it can be said that:

- **Ground-based**: Systems designed to be used from either stationary or mobile positions on the ground. This category includes systems installed on fixed sites, mobile systems, and systems mounted on ground vehicles.


- **Hand-held**: Systems that are designed to be operated by a single individual by hand. Many of these systems resemble rifles or other small arms.


- **UAV-based**: Systems designed to be mounted on drones, which can come into proximity with the targeted unmanned aircraft in order to employ interdiction elements at close range.

Once the database is analyzed some key data can be extracted:

 - A majority of the systems, 178 in total, are designed for ground-based use. Nineteen systems are designed to be handheld, and 19 systems are mounted aboard a drone. 
 
 
 - **Two** systems consist of a combination of ground-based and handheld elements, while one system consists of a combination of ground-based and drone-based elements.

In [16]:
platform_GROUND_BASED = platform.groupby('GROUND_BASED').size().reset_index(name='No. of GROUND_BASED')
platform_GROUND_BASED.columns = ['D.Opt', 'Num']
platform_UAV = platform.groupby('UAV').size().reset_index(name='No. of UAV')
platform_UAV.columns = ['D.Opt', 'Num']
platform_ALTRO = platform['ALTRO'].count()
df_app_ALTRO = pd.DataFrame({'D.Opt': ['handheld']})
df_app_ALTRO['Num'] = {platform_ALTRO}

df_app = platform_GROUND_BASED
df_app2 = df_app.append(platform_UAV)
df_app3 = df_app2.append(df_app_ALTRO)
#df_app6

fig = px.bar(df_app3, x='D.Opt', y='Num')

fig.update_layout(
    title="CSD - Counter Drone Systems // **Platform Systems**",
    xaxis_title="Platform Solution",
    yaxis_title="num. of Systems",
    font=dict(
        family="Courier New, monospace",
        size=14,
        color="#7f7f7f"
    )
)

fig.show()

### Relations among systems, solutions and options

#### Relations between Detection & Interdiction options

Most of the systems present only a detection solution (roughly sixty), followed by systems with only-interdiction solutions: with one option (roughly forty) and two options (roughly thirty) respectively. At least sixty-seven systems are advertised as being capable of both detection and interdiction.

In [17]:
data_DI = {'Detection': detection.count(axis = 1), 'Interdiction': interdiction.count(axis = 1)}
df_data_DI = pd.DataFrame(data_DI)
df_data_DI
# df che conta quante volte compare una coppia di valori
df_data_DI2 = pd.DataFrame(df_data_DI.groupby(list(df_data_DI.columns)).size(), columns=['Count']).reset_index()


fig = go.Figure(data=go.Scatter(
    x=df_data_DI2.Detection,
    y=df_data_DI2.Interdiction,
    mode='markers',
    marker=dict(size=df_data_DI2.Count*2, color=df_data_DI2.Count, colorscale='Viridis',showscale=True)
))


fig.update_layout(
    title="CSD - Counter Drone Systems // Relation Detection & Interdiction",
    xaxis_title="num. Detection options",
    yaxis_title="num. Interdiction options",
    font=dict(
        family="Courier New, monospace",
        size=14,
        color="#7f7f7f"
    )
)

fig.show()

#### Relation between Detection options & Platform types

Most of the systems present a solution with one detection option mounted in one platform type.

In [18]:
data_DP = {'Detection': detection.count(axis = 1), 'Platform':platform.count(axis = 1)}
df_data_DP = pd.DataFrame(data_DP)
# df che conta quante volte compare una coppia di valori
df_data_DP2 = pd.DataFrame(df_data_DP.groupby(list(df_data_DP.columns)).size(), columns=['Count']).reset_index()

fig = go.Figure(data=go.Scatter(
    x=df_data_DP2.Detection,
    y=df_data_DP2.Platform,
    mode='markers',
    marker=dict(size=df_data_DP2.Count*2, color=df_data_DP2.Count, colorscale='Viridis',showscale=True)
))

fig.update_layout(
    title="CSD - Counter Drone Systems // Relation Platform & Detection",
    xaxis_title="num. Detection options",
    yaxis_title="num. Platform options",
    font=dict(
        family="Courier New, monospace",
        size=14,
        color="#7f7f7f"
    )
)

fig.show()

#### Relation between Interdiction options & Platform types

Most of the systems present a solution with one interdiction option mounted in one platform type.

In [19]:
data_IP = {'Interdiction': interdiction.count(axis = 1), 'Platform':platform.count(axis = 1)}
df_data_IP = pd.DataFrame(data_IP)
# df che conta quante volte compare una coppia di valori
df_data_IP2 = pd.DataFrame(df_data_IP.groupby(list(df_data_IP.columns)).size(), columns=['Count']).reset_index()

fig = go.Figure(data=go.Scatter(
    x=df_data_IP2.Interdiction,
    y=df_data_IP2.Platform,
    mode='markers',
    marker=dict(size=df_data_IP2.Count*2, color=df_data_IP2.Count, colorscale='Viridis',showscale=True)
))

fig.update_layout(
    title="CSD - Counter Drone Systems // Relation Platform & Interdiction",
    xaxis_title="num. Interdiction opt",
    yaxis_title="num. Platform opt",
    font=dict(
        family="Courier New, monospace",
        size=12,
       #color="#7f7f7f"
    )
)

fig.show()

#### Relation between Detection & Interdiction on Platform type

Very few systems present a platform with detection & intediction options. Most of the systems are detection solution on one platform type.

In [20]:
data = {'Detection': detection.count(axis = 1), 'Interdiction': interdiction.count(axis = 1), 
        'Platform':platform.count(axis = 1)}
df_data = pd.DataFrame(data)
df_data2 = pd.DataFrame(df_data.groupby(list(df_data.columns)).size(), columns=['Count']).reset_index()

sz = (df_data2.Count*10)

#fig = px.scatter(df_data_DP2, x="Detection", y="Platform", color ="Count", size= sz)
#fig = px.scatter_3d(df_data2, x='Detection', y='Interdiction', z='Platform', size= sz , color= "Count")

fig = go.Figure(data=go.Scatter3d(
    x=df_data2.Detection,
    y=df_data2.Interdiction,
    z=df_data2.Platform,
    mode='markers',
    marker=dict(size=(df_data2.Count*2), color=df_data2.Count, colorscale='Viridis',showscale=True)
))

fig.update_layout(
    title="CSD - Counter Drone Systems",
    xaxis_title="num. Detection opt",
    yaxis_title="num. Platform opt",
    font=dict(
        family="Courier New, monospace",
        size=14,
        color="#7f7f7f"
    )
)

fig.show()

## Appendix

In [21]:
ff.create_table(table)